In [1]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [2]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [3]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [4]:
data[0]

'BV13S411A73f'

In [5]:
problems = []
for bv in data[:10]:
    problems.append(fetch_subtitle(bv))

Extracted segment: 13S411A73f
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/19058691521601443768adce9ddfec737839346a06b339dd1f2a?auth_key=1720505625-d6d9912bfc404fe38ddad16dc9f08861-0-050ec5a53bbf104b91a80f7221de8d38
Extracted segment: 1Jf421Q7dF
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/12057938431597546258b9bb9856015fcb212780f97e227e686e?auth_key=1720505629-34e8a32eeaf5499e835e739d0505ff7c-0-1dd6e2f669ea45d812ec69c9150c8974
Extracted segment: 1A1421k7JD
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/15559570961597535412b584fccc48b0e4d25ed9edcd9f4b72aa?auth_key=1720505633-6261e075bf4d400ea28dba6723e18b63-0-c106205e37cb608075ad57ae7fc01309
Extracted segment: 17z421z7fx
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/135595662015965277167223118aa

In [6]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [28]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意：对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ###讲解一:    ###讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    {model_expl}
"""

In [8]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
在一次羽毛球赛中，八个队进行循环赛，需要比赛几场？

### 讲解：
这八个队里边，如果你是其中的一个队，你需要和其他七个队进行比赛，所以是七场。然后我们需要计算所有队伍之间的比赛场次总和。可以用等差数列的求和公式来计算，即从7开始一直加到1，所以是7+6+5+4+3+2+1。我们可以用凑整法来快速计算：7+3=10，6+4=10，5+2=7，再加上1，总共是10+10+7+1=28场比赛。

### 相似题一：
在一次篮球赛中，十个队进行循环赛，需要比赛几场？

### 相似题一讲解：
这十个队里边，如果你是其中的一个队，你需要和其他九个队进行比赛，所以是九场。然后我们需要计算所有队伍之间的比赛场次总和。可以用等差数列的求和公式来计算，即从9开始一直加到1，所以是9+8+7+6+5+4+3+2+1。我们可以用凑整法来快速计算：9+1=10，8+2=10，7+3=10，6+4=10，再加上5，总共是10+10+10+10+5=45场比赛。executing
### 原题：
在一次足球比赛中，四个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）

### 讲解：
首先我们解释一下什么叫循环赛。比如说我们现在有三个球队，A、B、C。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；B和C也要进行一场比赛，这又算一场。那么每个队都赛了几场呢？A是两场，B也是两场，C也是两场。

但是在计算总场次的时候，我们要注意，A和B之间的比赛和B和A之间的比赛是同一场，所以不能重复计算。为了更清楚地理解，我们可以用线段图来表示。假设有四个队A、B、C、D，我们用线段连接每两个队，表示他们之间的一场比赛。

具体来说，A和B之间一场，A和C之间一场，A和D之间一场，共三场；然后B和C之间一场，B和D之间一场，共两场；最后C和D之间一场，共一场。这样总共是6场比赛。

所以，四个队进行循环赛需要比赛6场。

### 相似题一：
在一次篮球比赛中，五个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）

### 相似题一讲解：
首先我们解释一下什么叫循环赛。假设我们现在有五个球队，A、B、C、D、E。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；A和D再进行一场比赛，这又算一场；A和E再进行一场比赛，这又

In [9]:
data

['BV13S411A73f',
 'BV1Jf421Q7dF',
 'BV1A1421k7JD',
 'BV17z421z7fx',
 'BV1s6421g74i',
 'BV1rj421X7n7',
 'BV1Zv421C7rU',
 'BV1eu4m1P7ZD',
 'BV1nx4y127zz',
 'BV1tt421a7Ws',
 'BV11y421h7FQ',
 'BV1hC41147ca',
 'BV1gN4y1W7cX',
 'BV1ye411677L',
 'BV1tc411k7rf',
 'BV1Gj411s7Rd',
 'BV1Ec411U7yi',
 'BV1zb4y1g7QM',
 'BV1mw411W7pF',
 'BV1Vu4y1N7TE',
 'BV1mN411g73p',
 'BV1vc41197Ms',
 'BV1pN4y167Pc',
 'BV1Yj411e7rc',
 'BV1Ju4y1E7Gn',
 'BV1LG411C7FV',
 'BV1rC4y1n7rB',
 'BV1kQ4y1s7TJ',
 'BV1R94y1t7Lw',
 'BV1My4y1K7C3',
 'BV1mw411C74G',
 'BV11H4y1X7uo',
 'BV1sj411v7rH',
 'BV1ru4y1x7Y1',
 'BV1WH4y1f7kN',
 'BV1zB4y1f7My',
 'BV1L34y137qh',
 'BV1aH4y1d7BX',
 'BV12w411a7SR',
 'BV17V411c7s1',
 'BV1D94y1a7Yd',
 'BV16j41187hv',
 'BV1tp4y1F75H',
 'BV1H94y1a7VJ',
 'BV1yr4y197ZS',
 'BV1kj411y7CE',
 'BV1KG411o7Xf',
 'BV1ch4y1m7Tq',
 'BV1Uw411S7tx',
 'BV1Zh4y127K3',
 'BV1614y1y7js',
 'BV1Rm4y1T7Xm',
 'BV1sw411S7sc',
 'BV1er4y197y5',
 'BV1BF411k7tv',
 'BV1qz4y1M7a4',
 'BV12r4y1X7jG',
 'BV1Qp4y1P7cf',
 'BV1kG411Z7RK

In [10]:
extract_question_explanations(raw_problem_answer)

In [11]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '在一次足球比赛中，四个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）',
  'original_explanation': '首先我们解释一下什么叫循环赛。比如说我们现在有三个球队，A、B、C。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；B和C也要进行一场比赛，这又算一场。那么每个队都赛了几场呢？A是两场，B也是两场，C也是两场。\n\n但是在计算总场次的时候，我们要注意，A和B之间的比赛和B和A之间的比赛是同一场，所以不能重复计算。为了更清楚地理解，我们可以用线段图来表示。假设有四个队A、B、C、D，我们用线段连接每两个队，表示他们之间的一场比赛。\n\n具体来说，A和B之间一场，A和C之间一场，A和D之间一场，共三场；然后B和C之间一场，B和D之间一场，共两场；最后C和D之间一场，共一场。这样总共是6场比赛。\n\n所以，四个队进行循环赛需要比赛6场。',
  'similar_question_1': '在一次篮球比赛中，五个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）',
  'similar_explanation_1': '首先我们解释一下什么叫循环赛。假设我们现在有五个球队，A、B、C、D、E。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；A和D再进行一场比赛，这又算一场；A和E再进行一场比赛，这又算一场。接下来，B和C进行一场比赛，这算一场；B和D再进行一场比赛，这又算一场；B和E再进行一场比赛，这又算一场。然后，C和D进行一场比赛，这算一场；C和E再进行一场比赛，这又算一场。最后，D和E进行一场比赛，这算一场。'},
 {'original_question': '营业员有一张50元纸币，四张20元纸币，八张10元纸币。他要找给顾客90元钱，有几种找法？',
  'original_explanation': '这是关于我们人民币的组合问题。我们可以把这个问题列成表格来看一下。我们的纸币有三种：一种是50元，一种是20元，一种是10元。找钱的时候，我们应该先按大钱来找，因为这样比较方便。\n\n1. 先用一张50元纸币，那么还差40元。40元可以用两张20元纸币来找。\n2. 先用一张50元纸币，还差40元。40元可以用一张20元纸币和

In [12]:
model_explanation = ""
for ps in data:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n在一次足球比赛中，四个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）\n\n讲解:\n首先我们解释一下什么叫循环赛。比如说我们现在有三个球队，A、B、C。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；B和C也要进行一场比赛，这又算一场。那么每个队都赛了几场呢？A是两场，B也是两场，C也是两场。\n\n但是在计算总场次的时候，我们要注意，A和B之间的比赛和B和A之间的比赛是同一场，所以不能重复计算。为了更清楚地理解，我们可以用线段图来表示。假设有四个队A、B、C、D，我们用线段连接每两个队，表示他们之间的一场比赛。\n\n具体来说，A和B之间一场，A和C之间一场，A和D之间一场，共三场；然后B和C之间一场，B和D之间一场，共两场；最后C和D之间一场，共一场。这样总共是6场比赛。\n\n所以，四个队进行循环赛需要比赛6场。\n\n题目:\n在一次篮球比赛中，五个队进行循环赛，需要比赛多少场？（两个队之间比赛一次成一场）\n\n讲解:\n首先我们解释一下什么叫循环赛。假设我们现在有五个球队，A、B、C、D、E。A和B进行一场比赛，这算一场；A和C再进行一场比赛，这又算一场；A和D再进行一场比赛，这又算一场；A和E再进行一场比赛，这又算一场。接下来，B和C进行一场比赛，这算一场；B和D再进行一场比赛，这又算一场；B和E再进行一场比赛，这又算一场。然后，C和D进行一场比赛，这算一场；C和E再进行一场比赛，这又算一场。最后，D和E进行一场比赛，这算一场。\n\n题目:\n营业员有一张50元纸币，四张20元纸币，八张10元纸币。他要找给顾客90元钱，有几种找法？\n\n讲解:\n这是关于我们人民币的组合问题。我们可以把这个问题列成表格来看一下。我们的纸币有三种：一种是50元，一种是20元，一种是10元。找钱的时候，我们应该先按大钱来找，因为这样比较方便。\n\n1. 先用一张50元纸币，那么还差40元。40元可以用两张20元纸币来找。\n2. 先用一张50元纸币，还差40元。40元可以用一张20元纸币和两张10元纸币来找。\n3. 先用一张50元纸币，还差40元。40元可以用四张10元纸币来找。\n\n接下来，我们把50元纸币抛开，只用20元和10元纸币来找：\n\n4. 用四张20元纸币是80元，再加一张10元纸币是90元。\n5. 用

In [13]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [29]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig))
    pattern1 = r"###板书一:[\s\S]*?(?:###板书二:[\s\S]*?)?(?=###讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"###讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是组合数学中的组合问题，解题关键点是计算从n个元素中选出2个元素的组合数。

###板书一:
解：
$\because$ 五个队进行循环赛，每两个队之间比赛一次
$\therefore$ 需要计算从5个队中选出2个队的组合数
$C(5, 2) = \frac{5!}{2!(5-2)!}$

###讲解一:
首先，我们需要理解题目要求的比赛场次是每两个队之间比赛一次，这意味着我们需要计算从5个队中选出2个队的组合数。组合数的计算公式是$C(n, k) = \frac{n!}{k!(n-k)!}$，这里$n=5$，$k=2$。

###板书二:
$C(5, 2) = \frac{5!}{2! \cdot 3!}$
$= \frac{5 \times 4 \times 3!}{2 \times 1 \times 3!}$
$= \frac{5 \times 4}{2 \times 1}$
$= 10$

故答案是：10场比赛。

###讲解二:
接下来，我们将公式中的数值代入并进行计算。首先计算$5!$和$2! \cdot 3!$，然后进行约分，最后得到结果为10。因此，五个队进行循环赛需要比赛10场。executing
###讲解一:
首先，我们要理解题目中的循环赛是什么意思。循环赛是指每两个队之间都要进行一场比赛。也就是说，如果有五个队，每两个队之间都要比赛一次。

接下来，我们需要计算从5个队中选出2个队的组合数。这个问题可以用组合数学中的组合公式来解决。组合公式是用来计算从n个元素中选出k个元素的不同组合数，记作$C(n, k)$。

在这个题目中，n是5，k是2，所以我们需要计算$C(5, 2)$。根据组合公式，$C(n, k) = \frac{n!}{k!(n-k)!}$，其中$n!$表示n的阶乘。

因此，我们可以写出：
$C(5, 2) = \frac{5!}{2!(5-2)!}$

这一步是为了计算从5个队中选出2个队的所有可能组合数。接下来，我们会继续计算这个表达式的具体值。executing
知识点讲解:本题考查的知识点是组合问题，解题关键点是列出所有可能的组合并进行验证。

###板书一:
解：
设$x$为50元纸币的张数，$y$为20元纸币的张数，$z$为10元纸币的张数。
依题意得：
$50x +

In [30]:
with open("compare_results.md", 'w') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        # file.write("原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [31]:
print(len(orig_expls))

12


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

